In [1]:
import win32com.client
import ctypes
import numpy as np
import time

In [2]:
if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')

if win32com.client.Dispatch("CpUtil.CpCybos").IsConnect == 1:
        print('Creon Plus가 연결되었습니다.')
else:
        print('Creon Plus가 연결되지 않았습니다.')

정상: 관리자권한으로 실행된 프로세스입니다.
Creon Plus가 연결되었습니다.


In [3]:
objCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
KospiCodeList = objCpCodeMgr.GetStockListByMarket(1)

instMarketEye = win32com.client.Dispatch("CpSysDib.MarketEye")


In [4]:
def getScoreN(N,invest) :
    
    StockData = np.matrix(['Code', 'Name', '1/PER', 'ROA', '1/PBR', '1/PSR', 'GP/A', '1/(EV/EBIT)'])

    for i in range(len(KospiCodeList)):
        instMarketEye.SetInputValue(0, (5, 17, 20, 23, 67, 71, 75, 76, 77, 86, 88, 91))
        instMarketEye.SetInputValue(1, KospiCodeList[i])
        instMarketEye.BlockRequest()
        Code = KospiCodeList[i]                            # 종목코드
        OpP = instMarketEye.GetDataValue(0, 0)             # 시가
        Name = instMarketEye.GetDataValue(1, 0)            # 종목명
        StNum = instMarketEye.GetDataValue(2, 0)           # 상장주식수                              
        ClP = instMarketEye.GetDataValue(3, 0)             # 전일종가                                   
        PER = instMarketEye.GetDataValue(4, 0)             # PER
        CapSt = instMarketEye.GetDataValue(5, 0)           # 자본금
        DebtRat = instMarketEye.GetDataValue(6, 0)*0.01    # 부채비율
        ResRat = instMarketEye.GetDataValue(7, 0)*0.01     # 유보율
        ROE = instMarketEye.GetDataValue(8, 0)*0.01        # 자기자본이익률(ROE)
        SaleAcc = instMarketEye.GetDataValue(9, 0)         # 매출액
        NetInc = instMarketEye.GetDataValue(10, 0)         # 당기순이익
        OpProf = instMarketEye.GetDataValue(11, 0)         # 영업이익
    
        
        fStNum = float(StNum)                                   
        fClP = float(ClP)
        fCapSt = float(CapSt)*1000000
        fSaleAcc = float(SaleAcc)*1000000
        fNetInc = float(NetInc)
        fOpProf = float(OpProf)                                   
        fOpP = float(OpP)
        
    
        if objCpCodeMgr.IsBigListingStock(Code) :
            fStNum = fStNum*1000
            
        if fClP == 0:
            fClP = fOpP    # 종가 데이터가 없을 시 시가로 변환    
        
        if fNetInc*ROE*PER == 0 or DebtRat>=0.5 :
            pass
        else :
            PSR = fSaleAcc/(fStNum*fClP)
            PBR = fNetInc/(fStNum*fClP*ROE)
            ROA = ROE/(DebtRat+1)*100
            EV =  fStNum*fClP+fNetInc*DebtRat/ROE-ResRat*fCapSt
            
            if PSR < 0.2 or PBR <0.2 :
                pass
            else :
                rPSR = 1/PSR
                GPA = fOpProf*ROE/(fNetInc*(DebtRat+1))
                rPBR = 1/PBR
                GPA = fOpProf*ROE/(fNetInc*(DebtRat+1))
                rEVEBIT = fOpProf/EV
                           
                result = np.matrix([Code, Name,1/PER, ROA, rPBR, rPSR, GPA, rEVEBIT])
                StockData = np.vstack((StockData,result))
                #global Stock_data
                #Stock_data = StockData
                           
        time.sleep(0.25)
        
        tscore = np.matrix(np.zeros(len(StockData)-1)).T

    for i in range(6):
        data = StockData[1:,i+2].astype(np.float)
        datamean = np.mean(data)
        datastd = np.std(data)
        dataz = (data-datamean)/datastd
        datat = (dataz*10+50)/6
    
        tscore = np.add(datat,tscore).astype(np.float)
    
    score = np.hstack((StockData[1:,0],StockData[1:,1],tscore))

    tssort = np.sort(tscore,axis=0)[::-1]
    tssortN1 = tssort[N]
    rangeN = tscore>tssortN1

    code = StockData[1:,0]
    name = StockData[1:,1]

    tscoreN = tscore[rangeN].T
    codeN = code[rangeN].T
    nameN = name[rangeN].T
    
    tNsum = np.sum(tscoreN)
    tNratio = tscoreN/tNsum
    
    
    scoreN = np.hstack((codeN,nameN,tscoreN,tNratio*invest))
    #global ScoreN
    #ScoreN = scoreN
    print(scoreN)

In [5]:
#getScoreN(5,20)       # 실제
getScoreN(20,100)     # 모의

[['A000660' 'SK하이닉스' '70.4442041497052' '5.788603378842181']
 ['A002960' '한국쉘석유' '64.73093589493806' '5.319127652301302']
 ['A004450' '삼화왕관' '60.1014934163714' '4.938713014971644']
 ['A005500' '삼진제약' '61.4338545142743' '5.04819696821169']
 ['A005930' '삼성전자' '58.23698630335036' '4.785501089241428']
 ['A006650' '대한유화' '59.97972351085185' '4.928706830715041']
 ['A007700' 'F&F' '64.2392815607856' '5.278726997992693']
 ['A012750' '에스원' '57.371583602016386' '4.714388453908716']
 ['A020150' '일진머티리얼즈' '61.5818424237359' '5.060357561451082']
 ['A033780' 'KT&G' '58.42328942854033' '4.800810154238055']
 ['A035250' '강원랜드' '60.210797253608504' '4.947694826451926']
 ['A042700' '한미반도체' '56.610129684944496' '4.651817589912188']
 ['A057050' '현대홈쇼핑' '56.187481389088816' '4.617087360747376']
 ['A069260' '휴켐스' '58.34053748122146' '4.794010188113974']
 ['A078000' '텔코웨어' '55.925394914432594' '4.595550959405972']
 ['A114090' 'GKL' '59.94976250594415' '4.926244848550068']
 ['A120030' '조선선재' '57.50663885258191